# Imports and Setup

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import glob
import re
from datetime import datetime


# Models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Sklearn Tools
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression

# Setup plotting
sns.set_style("whitegrid")

# Create models folder if it doesn't exist
if not os.path.exists("../../models"):
    os.makedirs("../../models")

## 1. Target Definition & Cleaning
We are predicting **Overcost** (Market Efficiency).

* **Goal:** Predict if a campaign will be under or over budget based on its settings.
* **Formula:** `Overcost = approved_budget - media_cost_usd`
    * **Positive (+):** Under Budget (Good efficiency).
    * **Negative (-):** Over Budget (Bad efficiency).
* **Critical Step:** We MUST drop `approved_budget` and `media_cost_usd` from the input features (X). If we include them, the model will "cheat" by simply doing the subtraction itself (Data Leakage).

In [12]:
# FIND FILE AUTOMATICALLY
files = glob.glob("../Cleaned_Datasets/Marketing/*.csv")

if len(files) == 0:
    print("❌ Error: No files found. Check your directory path!")
else:
    print(f"Loading file: {files[0]}")
    df = pd.read_csv(files[0])

    # 1. CREATE TARGET (if not exists)
    if 'Overcost' not in df.columns:
        print("Creating 'Overcost' column...")
        budget_col = 'approved_budget' 
        cost_col = 'media_cost_usd'
        
        # Force numeric conversion to avoid string errors
        df[budget_col] = pd.to_numeric(df[budget_col], errors='coerce')
        df[cost_col] = pd.to_numeric(df[cost_col], errors='coerce')
        
        # Calculate Target
        df['Overcost'] = df[budget_col] - df[cost_col]

    TARGET = 'Overcost'

    # 2. DROP LEAKAGE & IDs
    leakage = ['approved_budget', 'campaign_budget_usd', 'media_cost_usd', 'campaign_item_id']
    df = df.drop([c for c in leakage if c in df.columns], axis=1)

    # Encode Categoricals
    df = pd.get_dummies(df, drop_first=True)
    df = df.fillna(0)

    print(f"Final Data Shape: {df.shape}")

Loading file: ../Cleaned_Datasets/Marketing\marketing_cleaned_prepared.csv
Creating 'Overcost' column...
Final Data Shape: (72206, 240)


## 2. Feature Selection
We analyze which campaign settings (Day, Category, Platform, etc.) have the biggest impact on the cost.
* **Visual Check:** Correlation Heatmap.
* **Statistical Check:** `SelectKBest` scores.

In [13]:
# --- CELL 5: FEATURE SELECTION (BASELINE = ALL) ---
import re

# 1. DEFINE X and Y
X = df.drop(TARGET, axis=1)
y = df[TARGET]

# 2. SANITIZE COLUMN NAMES
X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]
print("✅ Column names sanitized.")

# 3. SELECT FEATURES
print("Selecting ALL features for Baseline...")
selector = SelectKBest(score_func=f_regression, k='all') 
selector.fit(X, y)

selected_features = X.columns[selector.get_support()]
print(f"✅ Features ready: {len(selected_features)}")

✅ Column names sanitized.
Selecting ALL features for Baseline...
✅ Features ready: 239


## 3. Baseline Model Comparison
We train our three regressors to predict the `Overcost`.
* **Metric:** We use **RMSE (Root Mean Squared Error)**.
* **Interpretation:** Lower RMSE is better. It represents how many dollars ($) our prediction is off by on average.

In [ ]:
# --- CELL 7: TRAIN BASELINE MODELS ---
# Goal: Compare Train vs Test R2 to detect Overfitting

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'RandomForest': RandomForestRegressor(random_state=42), 
    'XGBoost': XGBRegressor(random_state=42, objective='reg:squarederror'),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1)
}

print(f"{'Model':<15} | {'Test R2':<10} | {'Train R2':<10} | {'RMSE':<10} | {'Gap (Train-Test)'}")
print("-" * 80)

for name, model in models.items():
    model.fit(X_train, y_train)
    
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train) 
    
    r2_test = r2_score(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    gap = r2_train - r2_test
    
    print(f"{name:<15} | {r2_test:.4f}     | {r2_train:.4f}     | {rmse:.4f}     | {gap:.4f}")

print("-" * 80)

Model           | Test R2    | Train R2   | RMSE       | Gap (Train-Test)
--------------------------------------------------------------------------------


## 4. Optimization & Saving
We take the winning model and tune its hyperparameters to see if we can reduce the error (RMSE) further.
Finally, we save the model using `joblib` with a versioned filename.

In [ ]:
# --- CELL 7: TRAIN BASELINE MODELS & FIND WINNER (MARKET) ---
# Goal: Compare Train vs Test R2 to detect Overfitting AND save the best model.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'RandomForest': RandomForestRegressor(random_state=42), 
    'XGBoost': XGBRegressor(random_state=42, objective='reg:squarederror'),
    'LightGBM': LGBMRegressor(random_state=42, verbose=-1)
}

# --- TRACKING VARIABLES ---
# For Market Cost, we want the LOWEST Error (RMSE), so we start with infinity
best_rmse = float('inf')
best_model_name = None
best_base_model = None

print(f"{'Model':<15} | {'Test R2':<10} | {'Train R2':<10} | {'RMSE':<10} | {'Gap (Train-Test)'}")
print("-" * 80)

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train) 
    
    # Calculate Metrics
    r2_test = r2_score(y_test, y_pred_test)
    r2_train = r2_score(y_train, y_pred_train)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    gap = r2_train - r2_test
    
    print(f"{name:<15} | {r2_test:.4f}     | {r2_train:.4f}     | {rmse:.4f}     | {gap:.4f}")
    
    # --- LOGIC TO SAVE THE WINNER (Based on Lowest RMSE) ---
    if rmse < best_rmse:
        best_rmse = rmse
        best_model_name = name
        best_base_model = model

print("-" * 80)
print(f"🏆 Winner: {best_model_name} (RMSE: {best_rmse:.4f})")



# 1. Setup Timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# 2. Check variables
if best_base_model is None:
    print("❌ Error: No model to save. Did Cell 7 run correctly?")
else:
    # 3. Create Filename
    # Using 'Exp1' to label it as Baseline
    # Using 'best_rmse' instead of 'opt_rmse'
    filename = f"Exp1_Market_{best_model_name}_RMSE-{best_rmse:.2f}_{timestamp}.joblib"
    
    # 4. Define Path
    save_dir = "../models"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    save_path = os.path.join(save_dir, filename)

    # 5. Save
    joblib.dump(best_base_model, save_path)
    print(f"✅ Baseline Market Model saved: {save_path}")

Model           | Test R2    | Train R2   | RMSE       | Gap (Train-Test)
--------------------------------------------------------------------------------
RandomForest    | 0.9534     | 0.9949     | 118623.2481     | 0.0416
XGBoost         | 0.7916     | 0.9481     | 250796.5554     | 0.1565
LightGBM        | 0.8270     | 0.9196     | 228474.4521     | 0.0926
--------------------------------------------------------------------------------
🏆 Winner: RandomForest (RMSE: 118623.2481)


NameError: name 'opt_rmse' is not defined